# 리팩토링
리팩토링은 소프트웨어 유지 관리에서 중요한 활동이지만 단위 텐스트가 없다면 정확성을 보장받기 어려움  
언제든 새로운 기능을 의도한 바대로 자원할 수 있어야 함  
이러한 요구사항을 수용할 수 있는 유일한 방법은 먼저 코드를 리팩토링하여 보다 일반적인 형태로 만들어야 함  

일반적으로 코드를 리팩토링할 때는 구조를 개선하여 보다 나은 코드로 만들려는 경우가 있고 좀 더 일반적인 코드로 수정하여 가독성을 높이려는 경우가 있음  
중요한 점은 이러한 **수정 작업 이전과 이후가 완전히 동일한 기능을 유지해야 한다는 것**임  
-> 리팩토링을 한 컴포넌트의 고객관점에서는 아무 일도 일어나지 않은 것처럼 느껴져야 함  

이전과 같은 기능을 지원할 때만 다른 코드를 사용할 수 있다는 것은 수정된 코드에 대해 회귀(regression) 테스트를 실행해야함을 의미  
회귀 테스트를 실행하는 유일한 효율적인 방법은 테스트를 자동화하는 것  
자동테스트의 가장 효율적인 버전이 바로 **단위 테스트**

## 코드의 진화  
이전 장의 예제에서 단위 테스트에서 제어할 수 없는 의존성이 있는 것들을 패치하여 코드의 부작용을 분리할 수 있었음  
이러한 종류의 작업에 mock.patch 함수를 사용하면 지시한 객체를 대체하여 Mock 객체를 돌려주기 때문에 편리하고 좋은 접근 방식임  

단점은 모듈을 포함하여 모의하려는 객체의 경로를 문자열로 제공해야 한다는 것임  
이 상태에서 코드를 리팩토링하면 패치한 모든 곳을 업데이트하거나 테스트가 실패할 것이므로 조금 취약한 부분임  

8_2의 예제에서 notify() 메서드가 구현 세부사항(requests 모듈)에 직접 의존한다는 것은 설계상의 문제임  
즉 앞서 언급한 취약성과 함께 단위 테스트에 영향을 미치고 있음  

이러한 메서드를 테스트 더블로 대체할 수 있지만 코드를 리팩토링하면 보다 나은 방법으로 이를 처리할 수 있음  
메서드들을 더 작은 메서드들로 나눈 후 의존성을 주입하여 의존성 역전 원칙을 적용해 requests 모듈이 제공하는 것과 같은 인터페이스를 지원하도록 하기  

In [3]:
from datetime import datetime

STATUS_ENDPOINT = "url.of.endpoint"

class BuildStatus: 
    endpoint = STATUS_ENDPOINT
    
    def __init__(self, transport):
        self.transport = transport
        
        @staticmethod
        def build_date() -> str:
            return datetime.utcnow().isoformat()
        
        def compose_payload(self, merge_request_id, status) -> dict:
            return {
                "id": merge_request_id,
                "status": status,
                "build_at": self.build_date()
            }
        
        def deliver(self, payload):
            response = self.transport.post(self.endpoint, json=payload)
            response.raise_for_status()
            return response
        
        def notify(self, merge_request_id, status):
            return self.deliver(self.compose_payload(merge_request_id, status))

위의 코드는 
- notify를 분리하여 compose와 deliver로 나누어 클래스를 패치하지 않고 바꿀 수 있도록 함
- compose_payload()라는 새로운 메서드를 만들고 transport라는 의존성을 주입할 것임  
- tranpsort는 주입되는 것이므로 테스트 더블의 형태를 변경하는 것이 훨씬 쉬워짐  

필요하다면 교체된 테스트 더블을 사용한 객체의 픽스처를 노출하는 것도 가능

In [6]:
import pytest 

@pytest.fixture
def build_status():
    bstatus = BuildStatus(Mock())
    bstatus.build_date = Mock(return_value="2018-01-01T00:00:01")
    return bstatus

def test_build_notification_sent(build_status):
    build_status.notify(1234, "OK")
    
    expected_payload = {
        "id": 1234,
        "status": "OK",
        "build_at": build_status.build_date()
    }
    
    build_status.transport.post.assert_called_with(build_status.endpoint, json=expected_payload)

## 상용 코드만 진화하는 것이 아니다
단위 테스트는 상용 코드만큼 중요한데 만약 상용 코드에 대해서 최대한 추상화 작업을 해야하는 것이라면, 단위테스트에도 그렇게 해야하는 것이 아닌 지 의문이 들 수 있음  
단위 테스트가 메인 코드만큼 중요하다면 확장성을 염두에 둬야하고, 가능한 유지보수가 가능하도록 디자인해야함  
모든 코드는 결국 원래의 작성자가 아닌 다른 엔지니어가 유지 관리해야 하는 것이므로 가독성이 높아야 함  

코드의 유연성을 높이기 위해 많은 관심을 기울이는 이유는 시간이 지남에 따라 요구사항이 변화하고 진화함을 알고 있기 때문  
또한 결국에는 도메인 비즈니스 규칙이 변경될 것이고 이러한 새로운 요구사항을 지원하기 위해 코드도 변경되어야 하기 때문  
상용 코드가 새로운 요구사항을 지원하도록 변경되었기 때문에 테스트 코드도 새로운 버전의 상용 코드를 지원하기 위해 변경되어야 함  

앞의 예제에서는 머지 리퀘스트 객체에 대한 일련의 테스트가 있었고, 다양한 조합으로 머지 리퀘스트의 상태를 확인했음  
이것은 좋은 방법이지만 더 좋은 방법이 있음  

문제를 정확히 이해하면 더 나은 추상화를 할 수 있고 특정 조건을 검사하는 더 높은 수준의 추상화를 만들 수 있음  
e.g > MergeRequest 클래스를 대상으로 하는 테스트 스위트 객체가 있는 경우, MergeRequest 클래스는 단일 책임 원칙(SRP)를 준수했을 것이므로 테스트는 이 클래스의 역할에만 초점을 맞추어 제작하면 됨  

이 테스트들은 해당 클래스에 대해서만 의미가 있지만 정형화된 단순 코드를 줄이는데 도움이 됨  
왜냐하면 해당 클래스가 정확하게 구현되었는지 반복해서 체크하는 대신 특정 기능을 캡슐화한 메서드를 만들고 여러 테스트에서 재사용할 수 있기 때문  

머지 리퀘스트의 상태를 확인하는 방법이 변경되거나 추가 검사를 하려는 경우 assert_approved() 메서드 하나만 변경하면 됨  
더욱 중요한 것은 일한 고차원의 추상화를 통해 단순한 단위테스트 코드가 자체 API 또는 도메인 특성을 진단해주는 테스트 프레임워크로 진화하여 테스트가 보다 선명해졌다는 점임  